<a href="https://colab.research.google.com/github/Yassmina-Abdo/realtime-head-pose-estimation/blob/main/Notebooks/read_extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import scipy.io
import numpy as np
import pandas as pd
import math
import glob
import os

## Read Matlab

In [ ]:
def read_matfile(path):
  mat = scipy.io.loadmat(path)
  #print(mat.keys())
  return mat

## Find Yaw Pitch Roll

In [ ]:
def get_ypr(mat):
  pose_para= mat['Pose_Para']
  pitch = pose_para[0][0] * 180 / math.pi
  yaw = pose_para[0][1] * 180 / math.pi
  roll = pose_para[0][2] * 180 / math.pi
  return np.array([yaw, pitch, roll],dtype=np.float32)

## Get XYZ of Nose From Yaw Pitch Roll

In [ ]:
def get_xyz(ypr):
  bins = np.array(range(-99, 102, 3))
  xyz =np.digitize([ypr[0], ypr[1], ypr[2]], bins) - 1
  return list(xyz)


## Get 68 points 

In [ ]:
def get_68pts(mat):
  pt3d_68 =np.array(np.transpose(mat['pt3d_68']).astype('float'),dtype=np.float32).reshape(-1,3)
  pts_x= pt3d_68[:,:-1][:,0]
  pts_y= pt3d_68[:,:-1][:,1]
  return list(pts_x),list(pts_y)


## Apply on all matlap files

In [ ]:
def apply_allfiles(path):
  xyzlist,yprlist,imgnolist,xslist,yslist=[],[],[],[],[]
  for file in glob.glob(path+'/*.mat'):
    #1. save mat file no
    img_no, _ = os.path.splitext(os.path.basename(file))
    imgnolist.append(img_no)
    #2. read mat file
    mat=read_matfile(file)
    #3. get yaw, pitch, roll
    ypr=get_ypr(mat)
    yprlist.append(ypr)
    #4. get x, y, z points from ypr
    xyzlist.append(get_xyz(ypr))
    #5. get 68 2d points
    pts_x,pts_y=get_68pts(mat)
    xslist.append(pts_x),yslist.append(pts_y)
  return xyzlist,yprlist,imgnolist,xslist,yslist

## Run

In [ ]:
# try one file
path='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Images/image00002.mat'
mat=read_matfile(path)
ypr=get_ypr(mat)
print(ypr)
xyz= get_xyz(ypr)
print(xyz)
pts_x,pts_y=get_68pts(mat)
print(pts_x)
print(pts_y)

[  1.044306  -22.874239    4.9088855]
[33, 25, 34]
[121.868034, 122.36761, 126.81924, 130.83179, 137.52313, 148.52173, 161.52802, 182.87668, 213.94006, 248.00537, 274.67615, 293.22473, 309.20724, 321.27457, 329.8783, 339.07126, 345.24066, 144.61288, 158.1359, 175.37297, 191.31126, 205.48387, 261.70496, 277.2635, 294.11136, 311.59164, 323.95364, 231.0957, 229.7904, 227.30476, 225.08842, 202.42142, 210.7933, 222.43806, 235.00792, 244.54094, 162.57912, 173.47615, 188.50098, 201.39299, 188.09201, 172.65819, 260.8649, 275.8604, 291.34073, 301.85168, 289.989, 273.8536, 180.50743, 192.32265, 209.19904, 218.46085, 228.46553, 248.99286, 261.81235, 243.97612, 228.25139, 212.72568, 199.55211, 189.6707, 183.1438, 204.32584, 216.49344, 229.86984, 259.4672, 227.68747, 213.52979, 201.8233]
[167.16422, 197.56262, 225.46965, 250.7149, 280.12787, 309.3874, 337.00238, 362.09814, 375.7305, 368.01422, 348.17438, 323.2857, 294.21448, 265.26617, 241.45877, 215.72095, 186.54291, 175.97385, 176.80212, 180.4087

In [ ]:
# run
path='/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Images'
xyzlist,yprlist,imgnolist,xslist,yslist= apply_allfiles(path)

In [ ]:
#save output--68 points file
colsx=  ['x' + str(i + 1) for i in range(68)]
colsy=  ['y' + str(i + 1) for i in range(68)]
dfx=pd.DataFrame(xslist,columns=colsx)
dfy=pd.DataFrame(yslist,columns=colsy)
df68 = pd.DataFrame(pd.concat([ dfx,dfy,pd.DataFrame(yprlist,columns=['yaw','pitch','roll']),pd.DataFrame(imgnolist,columns=['img']) ],axis=1))
df68.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/data68.csv',index=False)
df68.head(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,y63,y64,y65,y66,y67,y68,yaw,pitch,roll,img
0,279.052826,281.675476,285.686768,287.899536,284.693176,275.745636,261.695312,252.671860,252.653046,264.918823,...,309.331512,311.984436,318.515625,319.202942,317.465210,313.993927,69.062935,-48.795551,-31.882296,image03258
1,106.787560,107.119347,111.299889,115.847366,124.116203,140.815125,161.302399,185.516815,216.908142,245.068054,...,304.527283,305.674316,307.284821,319.076294,319.093506,316.208160,-13.392587,-6.686405,5.489456,image03247
2,123.746597,123.771423,123.467773,127.416306,139.268936,162.470306,190.685974,213.837158,222.323654,212.371353,...,316.445892,320.295105,330.613098,324.621582,320.403107,315.176819,-82.824692,-31.047533,61.351288,image03252
3,241.903885,241.499512,242.883118,242.789642,240.464447,236.202087,231.278656,229.640961,238.733490,256.186890,...,298.079254,300.004547,305.236816,304.317688,302.941620,300.311035,47.252384,-21.523932,0.100378,image03237
4,106.616531,113.848152,124.366989,134.661163,146.942078,164.044769,182.504761,206.717743,243.180405,279.360138,...,316.608398,315.733551,318.128784,319.939484,322.061279,323.020477,3.578812,5.349906,-7.209053,image03250


In [ ]:
#save output-- nose points
cols=['x','y','z','yaw','pitch','roll','img']
df_nose = pd.DataFrame(pd.concat([ pd.DataFrame(xyzlist),pd.DataFrame(yprlist),pd.DataFrame(imgnolist) ],axis=1))
df_nose.columns=cols
df_nose.to_csv('/content/drive/MyDrive/Head Pose Estimation ML Project/Dataset/Data/nose_data.csv',index=False)
df_nose.head(5)

,x,y,z,yaw,pitch,roll,img
0,56,16,22,69.062935,-48.795551,-31.882296,image03258
1,28,30,34,-13.392587,-6.686405,5.489456,image03247
2,5,22,53,-82.824692,-31.047533,61.351288,image03252
3,48,25,33,47.252384,-21.523932,0.100378,image03237
4,34,34,30,3.578812,5.349906,-7.209053,image03250
